In [1]:
#Importing the libraries
%matplotlib inline
import numpy as np
import pandas as pd
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from scipy.stats import zscore
from keras.callbacks import EarlyStopping
from matplotlib.pyplot import figure, show

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_excel('Volatile_gas.xlsx',sheet_name='Sheet1')
df.head()

,H2S,CO2,N2,C1,C2,C3,C4,C5,C6,C7+,MC7+,GC7,Temp,Pressure,CGR,SGOil,SGGas
0,0.00015,0.03735,0.01927,0.53774,0.08357,0.05518,0.04354,0.03472,0.02041,0.16807,239.40194,0.81068,889.17962,0.75017,0.68231,578.17637,998.24407
1,0.00038,0.02069,0.00981,0.52528,0.10473,0.04492,0.02863,0.03800,0.03238,0.19518,197.77196,0.80090,930.87734,0.73518,0.65770,562.14430,873.85808
2,0.00034,0.01870,0.03122,0.50508,0.13131,0.06494,0.04253,0.03776,0.02794,0.14019,176.69863,0.81296,650.74767,0.72381,0.69850,549.07451,660.55741
3,0.00032,0.03736,0.01403,0.58695,0.10961,0.05274,0.03391,0.01817,0.02473,0.12218,210.34309,0.83696,588.31389,0.74198,0.66499,544.80931,992.19413
4,0.00039,0.00925,0.03268,0.53156,0.09427,0.08539,0.03860,0.03512,0.03260,0.14015,188.31683,0.84668,798.11955,0.72758,0.64350,535.32295,853.04156


In [4]:
X = df.iloc[:, 12:]
y= df.iloc[:, :12]
y.info()

print(X)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16383 entries, 0 to 16382
Data columns (total 12 columns):
H2S     16383 non-null float64
CO2     16383 non-null float64
N2      16383 non-null float64
C1      16383 non-null float64
C2      16383 non-null float64
C3      16383 non-null float64
C4      16383 non-null float64
C5      16383 non-null float64
C6      16383 non-null float64
C7+     16383 non-null float64
MC7+    16383 non-null float64
GC7     16383 non-null float64
dtypes: float64(12)
memory usage: 1.5 MB
             Temp  Pressure      CGR      SGOil      SGGas
0       889.17962   0.75017  0.68231  578.17637  998.24407
1       930.87734   0.73518  0.65770  562.14430  873.85808
2       650.74767   0.72381  0.69850  549.07451  660.55741
3       588.31389   0.74198  0.66499  544.80931  992.19413
4       798.11955   0.72758  0.64350  535.32295  853.04156
5      1003.97036   0.74727  0.67274  556.23328  895.21733
6       637.51278   0.74199  0.66583  549.62868  829.49330
7     

In [5]:
#Dividing the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.6, test_size=0.4)

In [6]:
# define and fit the model
model = Sequential()
model.add(Dense(16, input_dim=5, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(12, kernel_initializer='normal'))
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=1000, callbacks=[earlyStopping], verbose=0)

C:\Anaconda\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [7]:
#predicting the test data
pred = model.predict(X_test)
print(pred[1])

[3.0926289e-04 1.3309065e-02 2.8428778e-02 6.1808169e-01 9.4363213e-02
 3.5792749e-02 3.3137564e-02 2.1419534e-02 2.0454153e-02 1.3841498e-01
 1.9650760e+02 8.4605891e-01]


In [8]:
#Using the rooted mean square error to evaluate the model's performance. 
score = np.sqrt(metrics.mean_squared_error(pred, y_test))
print('Score (RMSE):{}'.format(score))

Score (RMSE):9.063968377278423
